1. 확정 케이스 (3개)
확정_실패_케이스 (저장용/기증용/나이불명)


2. 확정_성공_가능_케이스 (최적 조건)


3. 고위험_케이스 (고령 + 다회시술 + 배아부족)2. 품질 점수들 (4개)
배아_품질_점수 (Day5 + 1-2개 + 효율)


5. 난자_품질_점수 (수집수 + 혼합비율)


6. 시술_품질_점수 (BLASTOCYST + ICSI)


7. 종합_임신_가능_점수 (나이 + 배아 + 시술)3. 과거 성공률 (4개)
IVF_과거_성공률


9. DI_과거_성공률


10. 전체_임신_성공률 (총 임신/총 시술)


11. 전체_출산_성공률 (총 출산/총 임신)4. 경과일 간격 (3개)
채취_혼합_간격 (난자 채취→혼합)


13. 혼합_이식_간격 (혼합→이식)


14. 채취_이식_총간격 (채취→이식)5. 배아 효율 심화 (3개)
배아_활용률 ((이식+저장)/생성)


16. 배아_선별률 ((생성-이식)/생성)


17. 미세주입_성공률 (미세주입 배아/난자)6. 교호작용 심화 (3개)
나이×배아수×Day5 (3-way)


19. 시술횟수×나이×불임원인


20. 고령×ICSI×다배아

In [1]:
import os
import numpy as np
import pandas as pd
from autogluon.tabular import TabularPredictor
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("AutoGluon Weekend Special - 35+ Features")
print("="*80)

# ============================================================
# 0) Load
# ============================================================
TRAIN_PATH = "../data/train.csv"
TEST_PATH  = "../data/test.csv"
SUB_PATH   = "../data/sample_submission.csv"

train = pd.read_csv(TRAIN_PATH)
test  = pd.read_csv(TEST_PATH)
sub   = pd.read_csv(SUB_PATH)

TARGET_COL = "임신 성공 여부"
ID_COL = "ID"

# ============================================================
# 1) Feature Engineering (35+ Features)
# ============================================================

# 고결측 변수 제거
HIGH_MISSING_COLS = [
    '난자 해동 경과일',           # 99.4%
    'PGS 시술 여부',             # 99.2%
    'PGD 시술 여부',             # 99.1%
    '착상 전 유전 검사 사용 여부', # 98.9%
    '임신 시도 또는 마지막 임신 경과 연수',  # 96.3%
]

EMBRYO_STAGE_COLS = [
    "단일 배아 이식 여부","착상 전 유전 진단 사용 여부","배아 생성 주요 이유",
    "총 생성 배아 수","미세주입된 난자 수","미세주입에서 생성된 배아 수",
    "이식된 배아 수","미세주입 배아 이식 수","저장된 배아 수",
    "미세주입 후 저장된 배아 수","해동된 배아 수","해동 난자 수",
    "수집된 신선 난자 수","저장된 신선 난자 수","혼합된 난자 수",
    "파트너 정자와 혼합된 난자 수","기증자 정자와 혼합된 난자 수",
    "동결 배아 사용 여부","신선 배아 사용 여부","기증 배아 사용 여부","대리모 여부",
]

INFERTILITY_COLS = [
    "남성 주 불임 원인","남성 부 불임 원인","여성 주 불임 원인","여성 부 불임 원인",
    "부부 주 불임 원인","부부 부 불임 원인","불명확 불임 원인",
    "불임 원인 - 난관 질환","불임 원인 - 남성 요인","불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인","불임 원인 - 자궁경부 문제","불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도","불임 원인 - 정자 면역학적 요인","불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태"
]

def safe_div(a, b):
    return np.where(b == 0, 0.0, a / b)

def preprocess(df):
    df = df.copy()
    
    # 고결측 제거
    df = df.drop(columns=HIGH_MISSING_COLS, errors='ignore')
    
    # === 기본 파생변수 (03번과 동일) ===
    
    # 1. 시술_대분류
    def major_procedure(x):
        if pd.isna(x): return "Unknown"
        x = str(x)
        if "IUI" in x: return "IUI"
        if "ICSI" in x: return "ICSI"
        if "IVF" in x: return "IVF"
        if "DI" in x: return "DI"
        return "Other"
    
    df["시술_대분류"] = df["특정 시술 유형"].apply(major_procedure)
    
    # 2. BLASTOCYST_포함
    s = df["특정 시술 유형"].astype("object").fillna("Unknown").astype(str)
    df["BLASTOCYST_포함"] = s.str.contains("BLASTOCYST", na=False).astype(int)
    df["AH_포함"] = s.str.contains("AH", na=False).astype(int)
    df["ICSI_포함"] = s.str.contains("ICSI", na=False).astype(int)
    
    # 3. 배아_이식_여부
    df["배아_stage_missing_count"] = df[EMBRYO_STAGE_COLS].isna().sum(axis=1)
    df["배아_이식_여부"] = (df["배아_stage_missing_count"] < len(EMBRYO_STAGE_COLS)).astype(int)
    
    # 4. 배아_진행_단계
    def embryo_stage(row):
        if row['배아_이식_여부'] == 0:
            return '배아단계_미도달'
        total = pd.to_numeric(row.get('총 생성 배아 수'), errors='coerce')
        implanted = pd.to_numeric(row.get('이식된 배아 수'), errors='coerce')
        if pd.isna(total) or total == 0:
            return '배아생성_실패'
        elif pd.isna(implanted) or implanted == 0:
            return '이식_미실시'
        else:
            return '이식_완료'
    
    df['배아_진행_단계'] = df.apply(embryo_stage, axis=1)
    
    # 5. 총시술_bin3
    def collapse_trials(x):
        if pd.isna(x): return "Unknown"
        if x == "0회": return "0회"
        if x in ["1회", "2회"]: return "1-2회"
        return "3회 이상"
    
    df["총시술_bin3"] = df["총 시술 횟수"].apply(collapse_trials)
    
    # 6. 나이_3구간
    def age_group_simple(age):
        if pd.isna(age) or age == "알 수 없음": return "Unknown"
        if age == "만18-34세": return "34세 이하"
        if age in ["만35-37세", "만38-39세"]: return "35-39세"
        return "40세 이상"
    
    df["나이_3구간"] = df["시술 당시 나이"].apply(age_group_simple)
    
    # 7. 이식배아_구간
    def embryo_count_bin(count):
        count = pd.to_numeric(count, errors='coerce')
        if pd.isna(count) or count == 0: return "0개"
        elif count <= 2: return "1-2개"
        else: return "3개 이상"
    
    df['이식배아_구간'] = df['이식된 배아 수'].apply(embryo_count_bin)
    
    # 8. Day5_이식_여부
    d = pd.to_numeric(df["배아 이식 경과일"], errors="coerce")
    df["Day5_이식_여부"] = (d == 5).astype(int)
    
    # 9. 불임원인_복잡도
    tmp = df[INFERTILITY_COLS].apply(pd.to_numeric, errors="coerce").fillna(0)
    df["불임_원인_개수"] = tmp.sum(axis=1)
    
    def infertility_complexity(count):
        if count == 0: return 'None'
        elif count == 1: return 'Single'
        elif count == 2: return 'Double'
        else: return 'Multiple'
    
    df['불임원인_복잡도'] = df['불임_원인_개수'].apply(infertility_complexity)
    
    # 10. 배아_해동_실시_여부
    df['배아_해동_실시_여부'] = df['배아 해동 경과일'].notna().astype(int)
    
    # 숫자형 안전 변환
    num_cols = [
        "총 생성 배아 수", "이식된 배아 수", "저장된 배아 수",
        "미세주입된 난자 수", "미세주입에서 생성된 배아 수",
        "수집된 신선 난자 수", "혼합된 난자 수",
        "배아 이식 경과일", "난자 채취 경과일", "난자 혼합 경과일"
    ]
    for c in num_cols:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    
    # 11-13. 비율 변수
    df["배아_생성_효율"] = safe_div(
        df["총 생성 배아 수"].fillna(0),
        df["수집된 신선 난자 수"].fillna(0)
    )
    
    df["배아_이식_비율"] = safe_div(
        df["이식된 배아 수"].fillna(0),
        df["총 생성 배아 수"].fillna(0)
    )
    
    df["배아_저장_비율"] = safe_div(
        df["저장된 배아 수"].fillna(0),
        df["총 생성 배아 수"].fillna(0)
    )
    
    # 14-15. 교호작용
    df['나이×Day5'] = df['시술 당시 나이'].astype(str) + '_' + df['Day5_이식_여부'].astype(str)
    df['시술횟수×나이'] = df['총시술_bin3'].astype(str) + '_' + df['나이_3구간'].astype(str)
    
    # === 🆕 추가 Feature 20개 ===
    
    # 1. 확정_실패_케이스
    storage_keywords = ['난자 저장용', '기증용', '배아 저장용', '연구용']
    df['확정_실패_케이스'] = 0
    if '배아 생성 주요 이유' in df.columns:
        df['확정_실패_케이스'] = df['배아 생성 주요 이유'].isin(storage_keywords).astype(int)
    if '시술 당시 나이' in df.columns:
        df['확정_실패_케이스'] = df['확정_실패_케이스'] | (df['시술 당시 나이'] == '알 수 없음').astype(int)
    
    # 2. 최적_이식_조건
    df['최적_이식_조건'] = (
        (df['Day5_이식_여부'] == 1) &
        (df['이식된 배아 수'].fillna(0).between(1, 2))
    ).astype(int)
    
    # 3. 고위험_케이스
    age_map = {'만40-42세': 1, '만43-44세': 1, '만45-50세': 1}
    df['고령_여부'] = df['시술 당시 나이'].map(age_map).fillna(0).astype(int)
    
    df['고위험_케이스'] = (
        (df['고령_여부'] == 1) &
        (df['총시술_bin3'] == '3회 이상') &
        (df['총 생성 배아 수'].fillna(0) < 3)
    ).astype(int)
    
    # 4. 배아_품질_점수
    df['배아_품질_점수'] = (
        (df['Day5_이식_여부'] * 3) +
        ((df['이식된 배아 수'].fillna(0).between(1, 2)).astype(int) * 2) +
        ((df['배아_생성_효율'].fillna(0) > 0.5).astype(int))
    )
    
    # 5. 난자_품질_점수
    df['난자_품질_점수'] = (
        ((df['수집된 신선 난자 수'].fillna(0) >= 10).astype(int) * 2) +
        ((df['배아_생성_효율'].fillna(0) > 0.6).astype(int))
    )
    
    # 6. 시술_품질_점수
    df['시술_품질_점수'] = (
        df['BLASTOCYST_포함'] * 2 +
        df['ICSI_포함'] * 1
    )
    
    # 7. 종합_임신_가능_점수
    df['종합_임신_가능_점수'] = (
        df['배아_품질_점수'] +
        df['난자_품질_점수'] +
        df['시술_품질_점수'] -
        (df['고령_여부'] * 2) -
        (df['고위험_케이스'] * 3)
    )
    
    # 8-11. 과거 성공률
    trials_map = {'0회': 0, '1회': 1, '2회': 2, '3회': 3, '4회': 4, '5회': 5, '6회 이상': 7}
    
    ivf_trials = df['IVF 시술 횟수'].map(trials_map).fillna(0)
    ivf_preg = df['IVF 임신 횟수'].map(trials_map).fillna(0)
    df['IVF_과거_성공률'] = safe_div(ivf_preg, ivf_trials)
    
    di_trials = df['DI 시술 횟수'].map(trials_map).fillna(0)
    di_preg = df['DI 임신 횟수'].map(trials_map).fillna(0)
    df['DI_과거_성공률'] = safe_div(di_preg, di_trials)
    
    total_trials = df['총 시술 횟수'].map(trials_map).fillna(0)
    total_preg = df['총 임신 횟수'].map(trials_map).fillna(0)
    df['전체_임신_성공률'] = safe_div(total_preg, total_trials)
    
    total_birth = df['총 출산 횟수'].map(trials_map).fillna(0)
    df['전체_출산_성공률'] = safe_div(total_birth, total_preg)
    
    # 12-14. 경과일 간격
    df['채취_혼합_간격'] = df['난자 혼합 경과일'].fillna(0) - df['난자 채취 경과일'].fillna(0)
    df['혼합_이식_간격'] = df['배아 이식 경과일'].fillna(0) - df['난자 혼합 경과일'].fillna(0)
    df['채취_이식_총간격'] = df['배아 이식 경과일'].fillna(0) - df['난자 채취 경과일'].fillna(0)
    
    # 15. 배아_활용률
    df['배아_활용률'] = safe_div(
        (df['이식된 배아 수'].fillna(0) + df['저장된 배아 수'].fillna(0)),
        df['총 생성 배아 수'].fillna(0)
    )
    
    # 16. 배아_선별률
    df['배아_선별률'] = safe_div(
        (df['총 생성 배아 수'].fillna(0) - df['이식된 배아 수'].fillna(0)),
        df['총 생성 배아 수'].fillna(0)
    )
    
    # 17. 미세주입_성공률
    df['미세주입_성공률'] = safe_div(
        df['미세주입에서 생성된 배아 수'].fillna(0),
        df['미세주입된 난자 수'].fillna(0)
    )
    
    # 18-20. 3-way 교호작용
    df['나이×배아수×Day5'] = (
        df['나이_3구간'].astype(str) + '_' +
        df['이식배아_구간'].astype(str) + '_' +
        df['Day5_이식_여부'].astype(str)
    )
    
    df['시술횟수×나이×불임'] = (
        df['총시술_bin3'].astype(str) + '_' +
        df['나이_3구간'].astype(str) + '_' +
        df['불임원인_복잡도'].astype(str)
    )
    
    df['고령×ICSI×다배아'] = (
        df['고령_여부'].astype(str) + '_' +
        df['ICSI_포함'].astype(str) + '_' +
        (df['이식된 배아 수'].fillna(0) >= 3).astype(int).astype(str)
    )
    
    return df

print("\n전처리 중...")
train_df = preprocess(train)
test_df = preprocess(test)

print(f"✅ Train shape: {train_df.shape}")
print(f"✅ Test shape: {test_df.shape}")

# ID 제거
train_data = train_df.drop(columns=[ID_COL, '배아_stage_missing_count'])
test_data = test_df.drop(columns=[ID_COL, '배아_stage_missing_count'])

print(f"\n최종 Feature 개수: {len(train_data.columns) - 1}개")

# ============================================================
# 2) AutoGluon 학습 (주말 스페셜 - 장시간)
# ============================================================
print("\n" + "="*80)
print("AutoGluon Training - Weekend Special")
print("="*80)
print("\n⏰ 예상 소요 시간: 5~8시간")
print("💾 메모리 사용량: 8-16GB 예상")
print("\n학습 시작...\n")

predictor = TabularPredictor(
    label=TARGET_COL,
    eval_metric='roc_auc',
    problem_type='binary',
    path='../outputs/autogluon_weekend_special'
)

predictor.fit(
    train_data=train_data,
    time_limit=28800,        # 8시간 (주말이니까!)
    presets='best_quality',
    num_bag_folds=10,        # 5→10 (더 안정적)
    num_bag_sets=2,          # 1→2 (더 다양)
    num_stack_levels=2,      # 1→2 (더 깊은 스태킹)
    hyperparameters='multimodal',  # 더 다양한 모델
    verbosity=2
)

# ============================================================
# 3) 결과 확인
# ============================================================
print("\n" + "="*80)
print("AutoGluon Results")
print("="*80)

leaderboard = predictor.leaderboard(train_data, silent=True)
print("\n=== Model Leaderboard (Top 20) ===")
print(leaderboard.head(20).to_string())

best_model = leaderboard.iloc[0]['model']
best_score = leaderboard.iloc[0]['score_val']
print(f"\n🏆 Best Model: {best_model}")
print(f"🎯 Best CV AUC: {best_score:.6f}")

print(f"\n📈 Previous Best (03번): 0.7407")
print(f"🚀 Improvement: +{best_score - 0.7407:.6f}")

# ============================================================
# 4) 예측 및 Submission
# ============================================================
print("\n" + "="*80)
print("Generating Predictions")
print("="*80)

test_predictions = predictor.predict_proba(test_data)

if isinstance(test_predictions, pd.DataFrame):
    test_proba = test_predictions[1].values
else:
    test_proba = test_predictions

# 🆕 Post-processing: 확정 실패 케이스 강제 0
mask = test_df['확정_실패_케이스'] == 1
if mask.sum() > 0:
    print(f"\n⚠️  확정 실패 케이스 {mask.sum()}개 → 강제로 0.0 설정")
    test_proba[mask] = 0.0

print(f"\n예측값 통계:")
print(f"Min:  {test_proba.min():.6f}")
print(f"Max:  {test_proba.max():.6f}")
print(f"Mean: {test_proba.mean():.6f}")
print(f"Std:  {test_proba.std():.6f}")

# Submission
sub['probability'] = test_proba
sub.to_csv("../outputs/07_submission_autogluon_weekend(0206).csv", index=False)

print("\n✅ Submission saved!")
print(f"   File: ../outputs/07_submission_autogluon_weekend(0206).csv")

# 상세 정보 저장
leaderboard.to_csv("../outputs/07_autogluon_weekend_leaderboard.csv", index=False)
print("✅ Leaderboard saved: ../07_outputs/autogluon_weekend_leaderboard.csv")

try:
    importance = predictor.feature_importance(train_data)
    importance.to_csv("../outputs/07_autogluon_weekend_feature_importance.csv")
    print("✅ Feature importance saved")
    print("\n=== Top 30 Features ===")
    print(importance.head(30).to_string())
except:
    print("Feature importance not available")

# 요약
summary = f"""
=== AutoGluon Weekend Special Summary ===

Total Features: {len(train_data.columns) - 1}
  - 원본: 64개 (고결측 5개 제거)
  - 파생: {len(train_data.columns) - 1 - 64}개

Best Model: {best_model}
CV AUC: {best_score:.6f}

Training Time: 8시간
Models Trained: {len(leaderboard)}

Top 5 Models:
"""

for i in range(min(5, len(leaderboard))):
    summary += f"  {i+1}. {leaderboard.iloc[i]['model']}: {leaderboard.iloc[i]['score_val']:.6f}\n"

summary += f"\nImprovement from 03번: +{best_score - 0.7407:.6f}"

print("\n" + "="*80)
print(summary)
print("="*80)

with open("../outputs/07_autogluon_weekend_summary.txt", "w") as f:
    f.write(summary)

print("\n✅ Summary saved: ../outputs/07_autogluon_weekend_summary.txt")
print("\n" + "="*80)
print("WEEKEND SPECIAL COMPLETE! 🎉")
print("="*80)

/opt/anaconda3/envs/fertility_ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


AutoGluon Weekend Special - 35+ Features

전처리 중...


Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=2, num_bag_folds=10, num_bag_sets=2
Dynamic stacking is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
Detecting stacked overfitting by sub-fitting AutoGluon on the input data. That is, copies of AutoGluon will be sub-fit on subset(s) of the data. Then, the holdout validation data is used to detect stacked overfitting.
Sub-fit(s) time limit is: 28800 seconds.
Starting holdout-based sub-fit for dynamic stacking. Context path is: ../outputs/autogluon_weekend_special/ds_sub_fit/sub_fit_ho.


✅ Train shape: (256351, 104)
✅ Test shape: (90067, 103)

최종 Feature 개수: 101개

AutoGluon Training - Weekend Special

⏰ 예상 소요 시간: 5~8시간
💾 메모리 사용량: 8-16GB 예상

학습 시작...



2026-02-06 19:13:42,274	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Running the sub-fit in a ray process to avoid memory leakage.
Spend 6645 seconds for the sub-fit(s) during dynamic stacking.
Time left for full fit of AutoGluon: 22155 seconds.
Starting full fit now with num_stack_levels 2.
Beginning AutoGluon training ... Time limit = 22155s
AutoGluon will save models to "../outputs/autogluon_weekend_special"
=================== System Info ===================
AutoGluon Version:  1.1.0
Python Version:     3.11.14
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:30:29 PDT 2025; root:xnu-11417.140.69~1/RELEASE_ARM64_T6000
CPU Count:          10
Memory Avail:       4.84 GB / 16.00 GB (30.2%)
Disk Space Avail:   215.51 GB / 460.43 GB (46.8%)
Train Data Rows:    256351
Train Data Columns: 101
Label Column:       임신 성공 여부
P


AutoGluon Results

=== Model Leaderboard (Top 20) ===
                    model  score_test  score_val eval_metric  pred_time_test  pred_time_val      fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    LightGBMLarge_BAG_L1    0.774003   0.738555     roc_auc       14.162534       5.977197     54.827851                14.162534                5.977197          54.827851            1       True          6
1    LightGBMLarge_BAG_L3    0.762386   0.739199     roc_auc      203.024789      44.941889   9471.225227                 9.423077                3.457149          38.608250            3       True         20
2    LightGBMLarge_BAG_L2    0.762203   0.739440     roc_auc      114.846477      28.013019   6864.321937                 9.976334                3.954448          50.494509            2       True         13
3       LightGBMXT_BAG_L3    0.757374   0.739274     roc_auc      199.486732      43.603050   9455.214053    

These features in provided data are not utilized by the predictor and will be ignored: ['불임 원인 - 여성 요인', '불임 원인 - 정자 농도', '불임 원인 - 정자 면역학적 요인', '불임 원인 - 정자 운동성', '불임 원인 - 정자 형태', '배아_이식_여부']



⚠️  확정 실패 케이스 4205개 → 강제로 0.0 설정

예측값 통계:
Min:  0.000000
Max:  0.742801
Mean: 0.258838
Std:  0.159289

✅ Submission saved!
   File: ../outputs/07_submission_autogluon_weekend(0206).csv
✅ Leaderboard saved: ../07_outputs/autogluon_weekend_leaderboard.csv


Computing feature importance via permutation shuffling for 95 features using 5000 rows with 5 shuffle sets...
	1270.16s	= Expected runtime (254.03s per shuffle set)
	887.85s	= Actual runtime (Completed 5 of 5 shuffle sets)


✅ Feature importance saved

=== Top 30 Features ===
               importance    stddev   p_value  n  p99_high   p99_low
이식된 배아 수         0.015188  0.003746  0.000410  5  0.022900  0.007475
이식배아_구간          0.015000  0.003622  0.000378  5  0.022458  0.007543
시술 당시 나이         0.008095  0.003016  0.001939  5  0.014306  0.001885
저장된 배아 수         0.004096  0.003161  0.022111  5  0.010603 -0.002412
시술 시기 코드         0.003289  0.000579  0.000111  5  0.004481  0.002096
나이_3구간           0.002990  0.002046  0.015431  5  0.007202 -0.001223
나이×배아수×Day5      0.002738  0.001918  0.016566  5  0.006686 -0.001210
난자 출처            0.002446  0.000771  0.001040  5  0.004033  0.000859
배아_저장_비율         0.002251  0.001885  0.027891  5  0.006133 -0.001630
나이×Day5          0.002159  0.001078  0.005494  5  0.004378 -0.000060
시술횟수×나이×불임       0.002061  0.000306  0.000057  5  0.002692  0.001430
수집된 신선 난자 수      0.002051  0.000853  0.002892  5  0.003807  0.000295
배아_생성_효율         0.001568  0.000334  0.000233  5  0